In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

## **SILVER LAYER SCRIPT**

### DATA ACCESS USING APP

In [0]:
# Configure OAuth for Azure Data Lake Storage Gen2 (ADLS Gen2) access.
# This is a secure template for publishing on GitHub.
# Replace the placeholder values in angle brackets `<...>` with your own information.

# Placeholder variables:
# <ADLS_ACCOUNT_NAME>: Your ADLS Gen2 account name (e.g., 'awspool2')
# <AZURE_CLIENT_ID>: The ID of your Service Principal (e.g., '36ff3ba8-...')
# <AZURE_TENANT_ID>: The ID of your Azure Active Directory Tenant (e.g., '8e5d54d1-...')

# *** SECURITY NOTE: ***
# The Client Secret MUST be stored in a Databricks Secret Scope.
# NEVER hard-code the Client Secret directly into your notebook.
# <SECRET_SCOPE_NAME>: The name of the Secret Scope you created (e.g., 'azure-pipeline-scope')
# <SECRET_KEY_NAME>: The key name for your Client Secret within the Scope (e.g., 'client-secret-key')

# 1. Retrieve the Client Secret from Databricks Secret Scope
client_secret = dbutils.secrets.get(scope="<SECRET_SCOPE_NAME>", key="<SECRET_KEY_NAME>")

# 2. Configure Spark
spark.conf.set("fs.azure.account.auth.type.<ADLS_ACCOUNT_NAME>.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.<ADLS_ACCOUNT_NAME>.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.<ADLS_ACCOUNT_NAME>.dfs.core.windows.net", "<AZURE_CLIENT_ID>")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.<ADLS_ACCOUNT_NAME>.dfs.core.windows.net", "https://login.microsoftonline.com/<AZURE_TENANT_ID>/oauth2/token")

# 3. Use the secure Client Secret
spark.conf.set("fs.azure.account.oauth2.client.secret.<ADLS_ACCOUNT_NAME>.dfs.core.windows.net", client_secret)

### DATA LOADING

**Reading Data**

In [0]:
df_cal = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("abfss://bronze@awspool2.dfs.core.windows.net/AdventureWorks_Calendar")

df_cus = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("abfss://bronze@awspool2.dfs.core.windows.net/AdventureWorks_Customers")

df_procat = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("abfss://bronze@awspool2.dfs.core.windows.net/AdventureWorks_Product_Categories")

df_pro = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("abfss://bronze@awspool2.dfs.core.windows.net/AdventureWorks_Products")

df_ret = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("abfss://bronze@awspool2.dfs.core.windows.net/AdventureWorks_Returns")

df_sales = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("abfss://bronze@awspool2.dfs.core.windows.net/AdventureWorks_Sales*")

df_ter = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("abfss://bronze@awspool2.dfs.core.windows.net/AdventureWorks_Territories")

df_subcat = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("abfss://bronze@awspool2.dfs.core.windows.net/Product_Subcategories")

### TRANSFORMATION

**Calendar**

In [0]:
df_cal = df_cal.withColumn("Month", month(df_cal.Date))\
                .withColumn("Year", year(df_cal.Date))
df_cal.limit(10).display()

Date,Month,Year
2015-01-01,1,2015
2015-01-02,1,2015
2015-01-03,1,2015
2015-01-04,1,2015
2015-01-05,1,2015
2015-01-06,1,2015
2015-01-07,1,2015
2015-01-08,1,2015
2015-01-09,1,2015
2015-01-10,1,2015


In [0]:
df_cal.write.format("parquet")\
    .mode("append")\
    .option("path","abfss://silver@awspool2.dfs.core.windows.net/AdventureWorks_Calendar")\
    .save()

**Customers**

In [0]:
df_cus.limit(10).display()

CustomerKey,Prefix,FirstName,LastName,BirthDate,MaritalStatus,Gender,EmailAddress,AnnualIncome,TotalChildren,EducationLevel,Occupation,HomeOwner
11000,MR.,JON,YANG,1966-04-08,M,M,jon24@adventure-works.com,"$90,000",2,Bachelors,Professional,Y
11001,MR.,EUGENE,HUANG,1965-05-14,S,M,eugene10@adventure-works.com,"$60,000",3,Bachelors,Professional,N
11002,MR.,RUBEN,TORRES,1965-08-12,M,M,ruben35@adventure-works.com,"$60,000",3,Bachelors,Professional,Y
11003,MS.,CHRISTY,ZHU,1968-02-15,S,F,christy12@adventure-works.com,"$70,000",0,Bachelors,Professional,N
11004,MRS.,ELIZABETH,JOHNSON,1968-08-08,S,F,elizabeth5@adventure-works.com,"$80,000",5,Bachelors,Professional,Y
11005,MR.,JULIO,RUIZ,1965-08-05,S,M,julio1@adventure-works.com,"$70,000",0,Bachelors,Professional,Y
11007,MR.,MARCO,MEHTA,1964-05-09,M,M,marco14@adventure-works.com,"$60,000",3,Bachelors,Professional,Y
11008,MRS.,ROBIN,VERHOFF,1964-07-07,S,F,rob4@adventure-works.com,"$60,000",4,Bachelors,Professional,Y
11009,MR.,SHANNON,CARLSON,1964-04-01,S,M,shannon38@adventure-works.com,"$70,000",0,Bachelors,Professional,N
11010,MS.,JACQUELYN,SUAREZ,1964-02-06,S,F,jacquelyn20@adventure-works.com,"$70,000",0,Bachelors,Professional,N


In [0]:
df_cus = df_cus.withColumn("fullName", concat_ws(' ',df_cus.Prefix, df_cus.FirstName, df_cus.LastName))
df_cus.limit(10).display()


CustomerKey,Prefix,FirstName,LastName,BirthDate,MaritalStatus,Gender,EmailAddress,AnnualIncome,TotalChildren,EducationLevel,Occupation,HomeOwner,fullName
11000,MR.,JON,YANG,1966-04-08,M,M,jon24@adventure-works.com,"$90,000",2,Bachelors,Professional,Y,MR. JON YANG
11001,MR.,EUGENE,HUANG,1965-05-14,S,M,eugene10@adventure-works.com,"$60,000",3,Bachelors,Professional,N,MR. EUGENE HUANG
11002,MR.,RUBEN,TORRES,1965-08-12,M,M,ruben35@adventure-works.com,"$60,000",3,Bachelors,Professional,Y,MR. RUBEN TORRES
11003,MS.,CHRISTY,ZHU,1968-02-15,S,F,christy12@adventure-works.com,"$70,000",0,Bachelors,Professional,N,MS. CHRISTY ZHU
11004,MRS.,ELIZABETH,JOHNSON,1968-08-08,S,F,elizabeth5@adventure-works.com,"$80,000",5,Bachelors,Professional,Y,MRS. ELIZABETH JOHNSON
11005,MR.,JULIO,RUIZ,1965-08-05,S,M,julio1@adventure-works.com,"$70,000",0,Bachelors,Professional,Y,MR. JULIO RUIZ
11007,MR.,MARCO,MEHTA,1964-05-09,M,M,marco14@adventure-works.com,"$60,000",3,Bachelors,Professional,Y,MR. MARCO MEHTA
11008,MRS.,ROBIN,VERHOFF,1964-07-07,S,F,rob4@adventure-works.com,"$60,000",4,Bachelors,Professional,Y,MRS. ROBIN VERHOFF
11009,MR.,SHANNON,CARLSON,1964-04-01,S,M,shannon38@adventure-works.com,"$70,000",0,Bachelors,Professional,N,MR. SHANNON CARLSON
11010,MS.,JACQUELYN,SUAREZ,1964-02-06,S,F,jacquelyn20@adventure-works.com,"$70,000",0,Bachelors,Professional,N,MS. JACQUELYN SUAREZ


In [0]:
df_cus.write.format("parquet")\
    .mode("append")\
    .option("path","abfss://silver@awspool2.dfs.core.windows.net/AdventureWorks_Customers")\
    .save()

**Sub Categories**

In [0]:
df_subcat.limit(10).display()

ProductSubcategoryKey,SubcategoryName,ProductCategoryKey
1,Mountain Bikes,1
2,Road Bikes,1
3,Touring Bikes,1
4,Handlebars,2
5,Bottom Brackets,2
6,Brakes,2
7,Chains,2
8,Cranksets,2
9,Derailleurs,2
10,Forks,2


In [0]:
df_subcat.write.format("parquet")\
    .mode("append")\
    .option("path","abfss://silver@awspool2.dfs.core.windows.net/AdventureWorks_SubCategories")\
    .save()

**Products**

In [0]:
df_pro.limit(10).display()

ProductKey,ProductSubcategoryKey,ProductSKU,ProductName,ModelName,ProductDescription,ProductColor,ProductSize,ProductStyle,ProductCost,ProductPrice
214,31,HL-U509-R,"Sport-100 Helmet, Red",Sport-100,"Universal fit, well-vented, lightweight , snap-on visor.",Red,0,0,13.0863,34.99
215,31,HL-U509,"Sport-100 Helmet, Black",Sport-100,"Universal fit, well-vented, lightweight , snap-on visor.",Black,0,0,12.0278,33.6442
218,23,SO-B909-M,"Mountain Bike Socks, M",Mountain Bike Socks,Combination of natural and synthetic fibers stays dry and provides just the right cushioning.,White,M,U,3.3963,9.5
219,23,SO-B909-L,"Mountain Bike Socks, L",Mountain Bike Socks,Combination of natural and synthetic fibers stays dry and provides just the right cushioning.,White,L,U,3.3963,9.5
220,31,HL-U509-B,"Sport-100 Helmet, Blue",Sport-100,"Universal fit, well-vented, lightweight , snap-on visor.",Blue,0,0,12.0278,33.6442
223,19,CA-1098,AWC Logo Cap,Cycling Cap,Traditional style with a flip-up brim; one-size fits all.,Multi,0,U,5.7052,8.6442
226,21,LJ-0192-S,"Long-Sleeve Logo Jersey, S",Long-Sleeve Logo Jersey,Unisex long-sleeve AWC logo microfiber cycling jersey,Multi,S,U,31.7244,48.0673
229,21,LJ-0192-M,"Long-Sleeve Logo Jersey, M",Long-Sleeve Logo Jersey,Unisex long-sleeve AWC logo microfiber cycling jersey,Multi,M,U,31.7244,48.0673
232,21,LJ-0192-L,"Long-Sleeve Logo Jersey, L",Long-Sleeve Logo Jersey,Unisex long-sleeve AWC logo microfiber cycling jersey,Multi,L,U,31.7244,48.0673
235,21,LJ-0192-X,"Long-Sleeve Logo Jersey, XL",Long-Sleeve Logo Jersey,Unisex long-sleeve AWC logo microfiber cycling jersey,Multi,XL,U,31.7244,48.0673


In [0]:
df_pro = df_pro.withColumn("ProductSKU",split(df_pro.ProductSKU,'-').getItem(0))\
                .withColumn("ProductName",split(df_pro.ProductName,' ').getItem(0))
df_pro.limit(10).display()

ProductKey,ProductSubcategoryKey,ProductSKU,ProductName,ModelName,ProductDescription,ProductColor,ProductSize,ProductStyle,ProductCost,ProductPrice
214,31,HL,Sport-100,Sport-100,"Universal fit, well-vented, lightweight , snap-on visor.",Red,0,0,13.0863,34.99
215,31,HL,Sport-100,Sport-100,"Universal fit, well-vented, lightweight , snap-on visor.",Black,0,0,12.0278,33.6442
218,23,SO,Mountain,Mountain Bike Socks,Combination of natural and synthetic fibers stays dry and provides just the right cushioning.,White,M,U,3.3963,9.5
219,23,SO,Mountain,Mountain Bike Socks,Combination of natural and synthetic fibers stays dry and provides just the right cushioning.,White,L,U,3.3963,9.5
220,31,HL,Sport-100,Sport-100,"Universal fit, well-vented, lightweight , snap-on visor.",Blue,0,0,12.0278,33.6442
223,19,CA,AWC,Cycling Cap,Traditional style with a flip-up brim; one-size fits all.,Multi,0,U,5.7052,8.6442
226,21,LJ,Long-Sleeve,Long-Sleeve Logo Jersey,Unisex long-sleeve AWC logo microfiber cycling jersey,Multi,S,U,31.7244,48.0673
229,21,LJ,Long-Sleeve,Long-Sleeve Logo Jersey,Unisex long-sleeve AWC logo microfiber cycling jersey,Multi,M,U,31.7244,48.0673
232,21,LJ,Long-Sleeve,Long-Sleeve Logo Jersey,Unisex long-sleeve AWC logo microfiber cycling jersey,Multi,L,U,31.7244,48.0673
235,21,LJ,Long-Sleeve,Long-Sleeve Logo Jersey,Unisex long-sleeve AWC logo microfiber cycling jersey,Multi,XL,U,31.7244,48.0673


In [0]:
df_pro.write.format("parquet")\
    .mode("append")\
    .option("path","abfss://silver@awspool2.dfs.core.windows.net/AdventureWorks_Products")\
    .save()

**Returns**

In [0]:
df_ret.limit(10).display()

ReturnDate,TerritoryKey,ProductKey,ReturnQuantity
2015-01-18,9,312,1
2015-01-18,10,310,1
2015-01-21,8,346,1
2015-01-22,4,311,1
2015-02-02,6,312,1
2015-02-15,1,312,1
2015-02-19,9,311,1
2015-02-24,8,314,1
2015-03-08,8,350,1
2015-03-13,9,350,1


In [0]:
df_ret.write.format("parquet")\
    .mode("append")\
    .option("path","abfss://silver@awspool2.dfs.core.windows.net/AdventureWorks_Returns")\
    .save()

**Territories**

In [0]:
df_ter.limit(10).display()

SalesTerritoryKey,Region,Country,Continent
1,Northwest,United States,North America
2,Northeast,United States,North America
3,Central,United States,North America
4,Southwest,United States,North America
5,Southeast,United States,North America
6,Canada,Canada,North America
7,France,France,Europe
8,Germany,Germany,Europe
9,Australia,Australia,Pacific
10,United Kingdom,United Kingdom,Europe


In [0]:
df_ter.write.format("parquet")\
    .mode("append")\
    .option("path","abfss://silver@awspool2.dfs.core.windows.net/AdventureWorks_Territories")\
    .save()

**Sales**

In [0]:
df_sales.limit(10).display()

OrderDate,StockDate,OrderNumber,ProductKey,CustomerKey,TerritoryKey,OrderLineItem,OrderQuantity
2017-01-01,2003-12-13,SO61285,529,23791,1,2,2
2017-01-01,2003-09-24,SO61285,214,23791,1,3,1
2017-01-01,2003-09-04,SO61285,540,23791,1,1,1
2017-01-01,2003-09-28,SO61301,529,16747,1,2,2
2017-01-01,2003-10-21,SO61301,377,16747,1,1,1
2017-01-01,2003-10-23,SO61301,540,16747,1,3,1
2017-01-01,2003-09-04,SO61269,215,11792,4,1,1
2017-01-01,2003-10-21,SO61269,229,11792,4,2,1
2017-01-01,2003-10-24,SO61286,528,11530,6,2,2
2017-01-01,2003-09-27,SO61286,536,11530,6,1,2


In [0]:
df_sales = df_sales.withColumn("StockDate",to_timestamp("StockDate"))\
                    .withColumn("OrderNumber",regexp_replace(df_sales.OrderNumber,'S','T'))\
                    .withColumn("Multiply",df_sales.OrderQuantity*df_sales.OrderLineItem)
df_sales = df_sales.withColumn("Multiply",df_sales.Multiply.cast("int"))

In [0]:
df_sales.limit(10).display()

OrderDate,StockDate,OrderNumber,ProductKey,CustomerKey,TerritoryKey,OrderLineItem,OrderQuantity,Multiply
2017-01-01,2003-12-13T00:00:00Z,TO61285,529,23791,1,2,2,4
2017-01-01,2003-09-24T00:00:00Z,TO61285,214,23791,1,3,1,3
2017-01-01,2003-09-04T00:00:00Z,TO61285,540,23791,1,1,1,1
2017-01-01,2003-09-28T00:00:00Z,TO61301,529,16747,1,2,2,4
2017-01-01,2003-10-21T00:00:00Z,TO61301,377,16747,1,1,1,1
2017-01-01,2003-10-23T00:00:00Z,TO61301,540,16747,1,3,1,3
2017-01-01,2003-09-04T00:00:00Z,TO61269,215,11792,4,1,1,1
2017-01-01,2003-10-21T00:00:00Z,TO61269,229,11792,4,2,1,2
2017-01-01,2003-10-24T00:00:00Z,TO61286,528,11530,6,2,2,4
2017-01-01,2003-09-27T00:00:00Z,TO61286,536,11530,6,1,2,2


In [0]:
df_sales.write.format("parquet")\
    .mode("append")\
    .option("path","abfss://silver@awspool2.dfs.core.windows.net/AdventureWorks_Sales")\
    .save()

**SALES ANALYSIS**

In [0]:
df_sales.groupBy("OrderDate").agg(count("OrderNumber").alias("TotalOrder")).display()

OrderDate,TotalOrder
2017-01-06,151
2017-01-27,142
2017-02-26,119
2017-01-24,173
2017-06-29,172
2017-02-16,124
2017-04-09,140
2017-02-28,162
2017-03-28,149
2017-06-30,136


Databricks visualization. Run in Databricks to view.

In [0]:
df_procat.limit(10).display()

ProductCategoryKey,CategoryName
1,Bikes
2,Components
3,Clothing
4,Accessories


In [0]:
df_subcat.createOrReplaceTempView("dim_product_subcategories")
df_pro.createOrReplaceTempView("dim_products")
df_sales.createOrReplaceTempView("fact_sales")

In [0]:
spark.sql('''
          select
          c.subcategoryname,
          count(distinct a.OrderNumber) as TotalSales

          from
            fact_sales a
          left join
            dim_products b
          on
            a.ProductKey=b.ProductKey
          left join
            dim_product_subcategories c
          on b.ProductSubcategoryKey=c.ProductSubcategoryKey
          group by c.subcategoryname
          ''').display()

subcategoryname,TotalSales
Mountain Bikes,4706
Hydration Packs,695
Road Bikes,7099
Bottles and Cages,4485
Vests,521
Helmets,6034
Jerseys,3113
Gloves,1332
Touring Bikes,2124
Bike Racks,302


Databricks visualization. Run in Databricks to view.